In [8]:
import numpy as np

# 입력 데이터를 한 번에 묶는 batch 처리 구현
x, t = get_test_data()
net_work = init_network()
batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(net_work, x_batch)
    # print("y_batch shape:", y_batch.shape)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum((p == t[i:i+batch_size]).numpy())
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


/var/folders/4g/wtxq72691_zfsk466n11xn780000gn/T/ipykernel_39464/3996949160.py:55: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  a1 = np.dot(x, W1) + b1


In [1]:
import torch
from torchvision import datasets, transforms


# PyTorch 기반 MNIST 데이터셋을 로드하는 함수
# tensor로 인해 자동 normalize가 되므로 알아서 전처리 해야됨
def load_mnist_torch(normalize=True, flatten=True, one_hot_label=False):
    """
    PyTorch 기반 MNIST 데이터셋을 (x_train, y_train), (x_test, y_test) 형태로 반환

    Parameters
    ----------
    normalize : True이면 0~1 정규화
    flatten : True이면 (N, 784), False이면 (N, 1, 28, 28)
    one_hot_label : True이면 라벨을 one-hot 인코딩으로 반환

    Returns
    -------
    (x_train, y_train), (x_test, y_test)
    """

    transform_list = []
    transform_list.append(transforms.ToTensor())  # 0~1 정규화 포함됨 (기본)
    transform = transforms.Compose(transform_list)

    train_dataset = datasets.MNIST(
        root="./data", train=True, download=True, transform=transform
    )
    test_dataset = datasets.MNIST(
        root="./data", train=False, download=True, transform=transform
    )

    x_train = torch.stack([data[0] for data in train_dataset])  # [60000, 1, 28, 28]
    y_train = torch.tensor([data[1] for data in train_dataset])  # [60000]

    x_test = torch.stack([data[0] for data in test_dataset])  # [10000, 1, 28, 28]
    y_test = torch.tensor([data[1] for data in test_dataset])  # [10000]

    if flatten:
        x_train = x_train.view(-1, 28 * 28)  # [60000, 784]
        x_test = x_test.view(-1, 28 * 28)  # [10000, 784]

    if one_hot_label:
        y_train = torch.nn.functional.one_hot(y_train, num_classes=10).float()
        y_test = torch.nn.functional.one_hot(y_test, num_classes=10).float()

    return (x_train, y_train), (x_test, y_test)

In [2]:
def sigmoid(x):
    """
    시그모이드 함수
    :param x: 입력값
    :return: 시그모이드 함수 적용 결과
    """
    return 1 / (1 + np.exp(-x))


def softmax(a):
    """
    소프트맥스 함수
    :param a: 입력값
    :return: 소프트맥스 함수 적용 결과
    """
    c = np.max(a)  # 최대값을 구함
    exp_a = np.exp(a - c)  # 최대값을 빼줌
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y


def get_test_data():
    """
    테스트 데이터셋을 반환
    """
    (x_train, y_train), (x_test, y_test) = load_mnist_torch(
        normalize=False, flatten=True, one_hot_label=False
    )
    return x_test, y_test


def init_network():
    """
    네트워크 초기화 함수.
    sample_weight.pkl 파일에서 네트워크 가중치를 로드하여 초기화
    :return: 초기화된 네트워크 딕셔너리
    """
    with open("sample_weight.pkl", "rb") as f:
        import pickle

        network = pickle.load(f)
    return network


def predict(network, x):
    """
    네트워크를 이용하여 입력 x에 대한 예측을 수행
    :param network: 초기화된 네트워크
    :param x: 입력 데이터
    :return: 예측 결과
    """
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    return y